In [ ]:
#@title Copyright 2020 The Earth Engine Community Authors { display-mode: "form" }
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [13]:
import ee
import os
import pandasa as pd
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=1UMpL_KieE0qOLcYstcg8aPA5CLr0h31Kl3m0dAU9Gk&tc=TbLr7S6psWNvSgsNHFYq9v0GZWb4blmaxJcQ2mLIR_c&cc=ZkJbhZt0g1iE8f_rQ7v2S9FBpk5xIbpIRtHmdl-Iwp4

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1ARtbsJpDnWW2AK1LyhYUyRXgvzzOhwzzDTjJfV6Ahw1ELbRl6RYjQKj-G1g

Successfully saved authorization token.


In [148]:
# Import the Landsat 7 Surface Temperature.
lst = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')
#lst=ee.ImageCollection('MODIS/061/MOD11A1')
#lst=ee.ImageCollection('MODIS/061/MYD11A1')
# Final date of interest (exclusive).
i_date = '2021-10-31'
#i_date='2000-02-24'
#i_date='2002-07-04'
#f_date='2022-10-09'
#f_date='2022-10-07'
f_date='2022-10-10'
banda='ST_B10'
#banda='LST_Day_1km'
# Define the urban location of interest as a point near Lyon, France.
u_lon = -69.8906
u_lat = -33.5894
u_poi = ee.Geometry.Point(u_lon,u_lat)
u_poits=ee.Geometry.MultiPoint([[-69.8906,-33.5894],[-70.8906,-34.5894]])
p1 = ee.Geometry.Point([-69.8906,-33.5894])
p2 = ee.Geometry.Point([-70.8906,-34.5894])

df_est=pd.read_excel(os.path.join('.','metadata_estaciones_glaciares.xlsx'))
print(df_est.columns)
features=[]
for index, row in df_est.iterrows():
#     print(dict(row))
#     construct the geometry from dataframe
    poi_geometry = ee.Geometry.Point([row['Lon'], row['Lat']])
#     print(poi_geometry)
#     construct the attributes (properties) for each point 
    poi_properties = dict(row)
#     construct feature combining geometry and properties
    poi_feature = ee.Feature(poi_geometry, poi_properties)
#     print(poi_feature)
    features.append(poi_feature)

# final Feature collection assembly
ee_fc = ee.FeatureCollection(features) 

print(ee_fc.getInfo())
scale=30

Index(['COD_BNA', 'NOMBRE', 'UTM_ESTE_W', 'UTM_NORTE_', 'HUSO', 'ALTITUD',
       'HABILITACI', 'ESTADO', 'COD_CUEN', 'COD_SUBC', 'NOM_CUEN', 'CINE_REG',
       'NOM_REG', 'CINE_PROV', 'NOM_PROV', 'CINE_COM', 'NOM_COM', 'Lon',
       'Lat'],
      dtype='object')
{'type': 'FeatureCollection', 'columns': {'ALTITUD': 'Integer', 'CINE_COM': 'Integer', 'CINE_PROV': 'Integer', 'CINE_REG': 'Integer', 'COD_BNA': 'String', 'COD_CUEN': 'Integer', 'COD_SUBC': 'Integer', 'ESTADO': 'String', 'HABILITACI': 'Object', 'HUSO': 'String', 'Lat': 'Float', 'Lon': 'Float', 'NOMBRE': 'String', 'NOM_COM': 'String', 'NOM_CUEN': 'String', 'NOM_PROV': 'String', 'NOM_REG': 'String', 'UTM_ESTE_W': 'Number', 'UTM_NORTE_': 'Number', 'system:index': 'String'}, 'features': [{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-69.72314252688268, -28.45701630154053]}, 'id': '0', 'properties': {'ALTITUD': 3985, 'CINE_COM': 3103, 'CINE_PROV': 31, 'CINE_REG': 3, 'COD_BNA': '03413001-9', 'COD_CUEN': 305, 'COD

In [154]:
def rasterExtraction(image):
    feature = image.sampleRegions(collection=ee_fc,scale=scale)
    return feature

def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('date').toInt())

def getCelsius(image):
    
    # LST
    lst = image.select(banda).multiply(0.00341802).add(149.0).add(-273.15)\
    .rename("LST")
    image = image.addBands(lst)

    return(image)

landsat_data = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2') \
    .filterDate(i_date,f_date).map(getCelsius).map(addDate)
  
results=landsat_data.filterBounds(ee_fc).select('LST').map(addDate)\
.map(rasterExtraction).flatten()
sample_result=results.first().getInfo()
#extract the properties column from feature collection
#column order may not be as out sample data order

columns=list(sample_result['properties'].keys())
#order data columns as per sample data order
#You can modify this for better optimizaction
column_df=list(df_est.columns)
column_df.extend(['LST','date'])
nested_list = results.reduceColumns(ee.Reducer.toList(len(column_df)),
                                    column_df).values().get(0)
data = nested_list.getInfo()
df = pd.DataFrame(data, columns=column_df)
df.index=pd.to_datetime(df['date'],format="%Y%m%d")
df=df[[x for x in df.columns if x!='date']]
df_pivot=pd.pivot_table(df,values='LST',index=df.index,columns=df['COD_BNA'])
print(df_pivot.head())
df_pivot.to_csv(os.path.join('.','results.csv'))

COD_BNA     03413001-9  04300004-7  05400004-9  05400005-7  05702011-3  \
date                                                                     
2021-11-04         NaN         NaN         NaN         NaN         NaN   
2021-11-09         NaN         NaN         NaN         NaN         NaN   
2021-11-19         NaN         NaN         NaN         NaN         NaN   
2021-11-24         NaN         NaN         NaN   21.960101   30.894805   
2021-12-06   35.245945    43.05612    46.42287   28.802977   45.253907   

COD_BNA     05702015-6  05703012-7  05703013-5  05703014-3  05703016-K  ...  \
date                                                                    ...   
2021-11-04         NaN         NaN         NaN         NaN         NaN  ...   
2021-11-09         NaN         NaN         NaN         NaN         NaN  ...   
2021-11-19         NaN         NaN         NaN         NaN         NaN  ...   
2021-11-24   -3.213616   14.795931    4.511109   29.913833  -12.845597  ...   
2021-12